**bold text**# Data on Tax Value of Adjudicated Properties

This data from BBR only contains information about: Vacant, City Owned - Owner Attributes, East Baton Rouge Redevelopment Authority Owned, Land Bank Available, Land Bank Acquisition Method, Land Bank Program Type, USPS Vacancy, Adjudicated Property.

In [15]:
import pandas as pd

# Mount drive so that we can import the same datasources
from google.colab import drive
drive.mount('/content/drive')

# Read in data from Tolemi and select only needed columns
data = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/tolemi-vacantuspslb.csv", skiprows=4)
data['Parcel ID'] = data['Parcel ID'].map(lambda x: x.lstrip('="').rstrip('"'))
selcol = ['Parcel ID', 'Address', 'Current Use', 'Total Assessed Value',
       'Blight Survey - Condition', 'Homestead Exemption',
       'City Owned - Owner Attributes',
       'East Baton Rouge Redevelopment Authority Owned',
       'Land Bank Acquisition Method', 'Land Bank Program Type',
       'Land Bank Available', 'Is Vacant Property', 'USPS Vacancy',
       'Earliest Adjudication Date', 'Adjudicated Property', 'Adjudicated Property - Count']
data = data[selcol]
data['Parcel ID'] = data['Parcel ID'].apply(str)
data['Parcel ID'] = data['Parcel ID'].str.zfill(10)

# Read in tax data downloaded from the East Baton Rouge Tax Roll
taxes = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/EBRP_Tax_Roll.csv")

# Change the data type to string and pad the Parcel ID to normalize across datasets
taxes['PROPERTY NUMBER'] = taxes['PROPERTY NUMBER'].apply(str)
taxes['PROPERTY NUMBER'] = taxes['PROPERTY NUMBER'].str.zfill(10)

# Merge the two datasets on the Property ID and Parcel ID
merged = pd.merge(taxes, data, right_on= 'Parcel ID', left_on= 'PROPERTY NUMBER')

# Drop unneeded columns from merged dataset
selcol = ['Parcel ID', 'Adjudicated Property', 'TAX YEAR', 'Current Use', 'Total Assessed Value']
mergedselect = merged[selcol]

# Run in place calculation on each line that creates a new column for both Conservative and All In Tax estimates
mergedselect = mergedselect.assign(ConservativeTax = lambda x: (x['Total Assessed Value'] * .052733))
mergedselect = mergedselect.assign(AllInTax = lambda x: (x['Total Assessed Value'] * .108803))

# Set index to the Parcel ID
mergedselect = mergedselect[mergedselect['Adjudicated Property'] == 'Yes']

# Create pivot table that has one row per property ID with years as columns
df_taxpivotvalues = mergedselect.pivot_table(values=['Total Assessed Value','ConservativeTax','AllInTax'], index='Parcel ID', columns='TAX YEAR', fill_value = 0, aggfunc= ['sum'], margins = True, margins_name='Total')
df_taxpivotvalues

# Pandas Pivot table to CSV file for merging with call data
df_taxpivotvalues.to_csv('/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/TaxCalculation.csv',index=True)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-15-e776d4bcf175>:22: DtypeWarning: Columns (9,20,28) have mixed types. Specify dtype option on import or set low_memory=False.
  taxes = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/EBRP_Tax_Roll.csv")


In [14]:
# Pandas Pivot table to CSV file
df_taxpivotvalues.to_csv('/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/TaxCalculation.csv',index=True)

In [ ]:
# Convert the pivot table to a DataFrame
df_taxpivot = df_taxpivotvalues.reset_index()
df_taxpivot

Parcel ID           sum                                            \
                          AllInTax                                             
TAX YEAR                      2015          2016          2017          2018   
0         0000000647      59.84165      59.84165      59.84165      59.84165   
1         0000003158      54.40150      54.40150      54.40150      54.40150   
2         0000003247      60.92968      60.92968      30.46484      30.46484   
3         0000003298      35.90499      35.90499      35.90499      35.90499   
4         0000003670     108.80300     108.80300     108.80300     108.80300   
..               ...           ...           ...           ...           ...   
599       0003261956      23.93666      23.93666      23.93666      23.93666   
600       0003262073      42.43317      42.43317      42.43317      42.43317   
601       0003588408      95.74664      95.74664      95.74664      95.74664   
602       0003588939     108.80300     108.80300     108.80300     108.80300   
603            Total  161140.50709  160991.44698  160347.33322  159859.89578   

                                                                  \
                                                                   
TAX YEAR          2019          2020          2021          2022   
0             59.84165      59.84165      59.84165      59.84165   
1             54.40150      54.40150      54.40150      54.40150   
2             30.46484      30.46484      30.46484      30.46484   
3             35.90499      35.90499      35.90499      35.90499   
4            108.80300     108.80300     108.80300     108.80300   
..                 ...           ...           ...           ...   
599           23.93666      23.93666      23.93666      23.93666   
600           42.43317      42.43317      42.43317      42.43317   
601           95.74664      95.74664      95.74664      95.74664   
602          108.80300     108.80300     108.80300     108.80300   
603       159607.47282  159009.05632  158702.23186  158555.34781   

                        ...                                                \
                        ... ConservativeTax Total Assessed Value            
TAX YEAR         Total  ...           Total                 2015     2016   
0         4.787332e+02  ...       232.02520                  550      550   
1         4.352120e+02  ...       210.93200                  500      500   
2         3.046484e+02  ...       147.65240                  560      560   
3         2.872399e+02  ...       139.21512                  330      330   
4         8.704240e+02  ...       421.86400                 1000     1000   
..                 ...  ...             ...                  ...      ...   
599       1.914933e+02  ...        92.81008                  220      220   
600       3.394654e+02  ...       164.52696                  390      390   
601       7.659731e+02  ...       371.24032                  880      880   
602       8.704240e+02  ...       421.86400                 1000     1000   
603       1.278213e+06  ...    619505.17468              1481030  1479660   

                                                                            
                                                                            
TAX YEAR     2017     2018     2019     2020     2021     2022       Total  
0             550      550      550      550      550      550      4400.0  
1             500      500      500      500      500      500      4000.0  
2             280      280      280      280      280      280      2800.0  
3             330      330      330      330      330      330      2640.0  
4            1000     1000     1000     1000     1000     1000      8000.0  
..            ...      ...      ...      ...      ...      ...         ...  
599           220      220      220      220      220      220      1760.0  
600           390      390      390      390      390      390      3120.0  
601        

In [ ]:
# Read in data from Tolemi and select only needed columns
data = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/tolemi-vacantuspslb.csv", skiprows=4)
data['Parcel ID'] = data['Parcel ID'].map(lambda x: x.lstrip('="').rstrip('"'))
selcol = ['Parcel ID', 'Address', 'Current Use', 'Total Assessed Value',
       'Blight Survey - Condition', 'Homestead Exemption',
       'City Owned - Owner Attributes',
       'East Baton Rouge Redevelopment Authority Owned',
       'Land Bank Acquisition Method', 'Land Bank Program Type',
       'Land Bank Available', 'Is Vacant Property', 'USPS Vacancy',
       'Earliest Adjudication Date', 'Adjudicated Property', 'Adjudicated Property - Count']
data = data[selcol]
# Normalize Parcel ID to 10 digits
data['Parcel ID'] = data['Parcel ID'].apply(str)
data['Parcel ID'] = data['Parcel ID'].str.zfill(10)
data



,Parcel ID,Address,Current Use,Total Assessed Value,Blight Survey - Condition,Homestead Exemption,City Owned - Owner Attributes,East Baton Rouge Redevelopment Authority Owned,Land Bank Acquisition Method,Land Bank Program Type,Land Bank Available,Is Vacant Property,USPS Vacancy,Earliest Adjudication Date,Adjudicated Property,Adjudicated Property - Count
0,0000000000,"Address Unknown, Baton Rouge, LA",VACANT,NaN,NaN,No,No,No,NaN,NaN,NaN,No,NaN,NaN,Unknown,NaN
1,0000000000,"4133 TUSCARORA ST, Baton Rouge, LA",VACANT,NaN,NaN,No,No,No,NaN,NaN,NaN,No,NaN,NaN,Unknown,NaN
2,0000000000,"1200 PLANK RD, Baton Rouge, LA",VACANT,NaN,NaN,No,No,No,NaN,NaN,NaN,No,NaN,NaN,Unknown,NaN
3,0000000000,"3900 NORTHSHORE AVE, Baton Rouge, LA",VACANT,NaN,NaN,No,No,No,NaN,NaN,NaN,No,NaN,NaN,Unknown,NaN
4,0000000000,"Address Unknown, Baton Rouge, LA",VACANT,NaN,NaN,No,No,No,NaN,NaN,NaN,No,NaN,NaN,Unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,0000098965,"615 IBERVILLE ST, Baton Rouge, LA",VACANT,630.0,NaN,No,No,No,NaN,NaN,NaN,No,Unavailable,NaN,Yes,9.0
2430,0000099481,"2240 ADELLA ST, Baton Rouge, LA",VACANT,280.0,NaN,No,No,No,NaN,NaN,NaN,No,Unavailable,NaN,Unknown,NaN
2431,0000099503,"Address Unknown, Baton Rouge, LA",VACANT,170.0,NaN,No,No,No,NaN,NaN,NaN,No,Unavailable,2012-12-31,Yes,11.0
2432,0000996637,"8552 PARIS AVE, Baton Rouge, LA",VACANT,18030.0,NaN,No,No,No,NaN,NaN,NaN,No,Not Vacant,NaN,Unknown,NaN


In [ ]:
# Read in tax data downloaded from the East Baton Rouge Tax Roll
taxes = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/EBRP_Tax_Roll.csv")

# Change the data type to string and pad the Parcel ID to normalize across datasets
taxes['PROPERTY NUMBER'] = taxes['PROPERTY NUMBER'].apply(str)
taxes['PROPERTY NUMBER'] = taxes['PROPERTY NUMBER'].str.zfill(10)
taxes

<ipython-input-3-569f3d731eda>:2: DtypeWarning: Columns (9,20,28) have mixed types. Specify dtype option on import or set low_memory=False.
  taxes = pd.read_csv("/content/drive/MyDrive/Capstone Baton Rouge/Working documents/Data Analysis/Data Sets/EBRP_Tax_Roll.csv")


,TAX YEAR,PROPERTY NUMBER,LEGACY PROPERTY NUMBER,ASSESSMENT TYPE,ASSESSMENT STATUS,TAXPAYER NAME,TAXPAYER ADDRESS,TAXPAYER SUBADDRESS,TAXPAYER CITY STATE ZIP,WARD,...,TRANSFER DATE,DEED NO,LEGAL DESCRIPTION,RESTORATION TAX ABATEMENT,HOMESTEAD EXEMPTION,FAIR MARKET VALUE,TOTAL ASSESSED VALUE,HOMESTEAD EXEMPTION VALUE,TAXABLE PARISH,TAX FREEZE
0,2015,0000000019,000-0001-9,REAL PROPERTY,ACTUAL,LA CAPITOL FEDERAL CREDIT UNION,700 MAIN ST,NaN,BATON ROUGE LA 70802,1,...,12/11/2002,11410787,"Ward 1-1 #4408, Lot: 5 PT, Square: 12, Subdiv:...",NO,NO,37500.0,3750.0,0.0,3750.0,NaN
1,2015,0000000027,000-0002-7,REAL PROPERTY,ACTUAL,"ROBINSON, GILBERT L.; WILLARD C.",C/O GLENN ROBINSON,4388 WELLS ST.,BATON ROUGE LA 70808,1,...,01/01/1977,91940451,"Ward 1-1 #6797, Lot: 17, Square: 344, Subdiv: ...",NO,NO,3500.0,350.0,0.0,350.0,NaN
2,2015,0000000035,000-0003-5,REAL PROPERTY,ACTUAL,"BURRELL, KENNY",6825 CALUMET ST.,NaN,BATON ROUGE LA 70805,1,...,04/14/2004,11590735,"Ward 1-1 #1007, Lot: 11-B, Square: 6, Subdiv: ...",NO,NO,2000.0,200.0,0.0,200.0,NaN
3,2015,0000000035,000-0003-5,REAL PROPERTY,ACTUAL,"BURRELL, KENNY",6825 CALUMET ST.,NaN,BATON ROUGE LA 70805,1,...,04/14/2004,11590735,"Ward 1-1 #1007, Lot: 11-B, Square: 6, Subdiv: ...",NO,NO,6000.0,600.0,0.0,600.0,NaN
4,2015,0000000043,000-0004-3,REAL PROPERTY,ACTUAL,"ANDRE, SHANA B. & CASEY LYNN",1958 WISTERIA ST.,NaN,BATON ROUGE LA 70806,1,...,06/07/2010,12240692,"Ward 1-1 #158, Lot: 12, Square: 5, Subdiv: ROS...",NO,YES,162000.0,16200.0,7500.0,8700.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899771,2022,0000999903,009-9990-3,REAL PROPERTY,ACTUAL,"HIXSON, MICHAEL",19544 ARCADIAN SHORES AVE,NaN,BATON ROUGE LA 70809,3-0,...,03/11/2022,172-1317,"Ward 3-0, Lot:196, Subdiv:SANTA MARIA. 2ND FI...",NO,YES,100000.0,10000.0,0.0,10000.0,NONE
2899772,2022,0000999903,009-9990-3,REAL PROPERTY,ACTUAL,"HIXSON, MICHAEL",19544 ARCADIAN SHORES AVE,NaN,BATON ROUGE LA 70809,3-0,...,03/11/2022,172-1317,"Ward 3-0, Lot:196, Subdiv:SANTA MARIA. 2ND FI...",NO,YES,549000.0,54900.0,7500.0,47400.0,NONE
2899773,2022,0000999946,009-9994-6,REAL PROPERTY,ACTUAL,"ROGERS, ROBERT D",5634 MADISON AVE,NaN,BATON ROUGE LA 70806,2-3,...,11/22/2017,151-1285,"Ward 2-3, Lot:D-3, Subdiv:JOHNSON, JULIUS ESTA...",NO,NO,20400.0,2040.0,0.0,2040.0,NONE
2899774,2022,0000099996,000-9999-6,REAL PROPERTY,ACTUAL,"JAMES, LARRY DIONELL",10724 TIMBERLANE AVE,NaN,BATON ROUGE LA 70811,1-1,...,12/17/2013,303-1254,"Ward 1-1, Lot:16, Square:13, Subdiv:ISTROUMA. ...",NO,NO,30800.0,3080.0,0.0,3080.0,NONE


In [ ]:
# Merge the two datasets on the Property ID and Parcel ID
merged = pd.merge(taxes, data, right_on= 'Parcel ID', left_on= 'PROPERTY NUMBER')

# Drop unneeded columns from merged dataset
selcol = ['Parcel ID', 'Adjudicated Property', 'TAX YEAR', 'Current Use', 'Total Assessed Value']
mergedselect = merged[selcol]

# Run in place calculation on each line that creates a new column for both Conservative and All In Tax estimates
mergedselect = mergedselect.assign(ConservativeTax = lambda x: (x['Total Assessed Value'] * .052733))
mergedselect = mergedselect.assign(AllInTax = lambda x: (x['Total Assessed Value'] * .108803))



In [ ]:
# Set index to the Parcel ID
mergedselect = mergedselect[mergedselect['Adjudicated Property'] == 'Yes']

mergedselect

,Parcel ID,Adjudicated Property,TAX YEAR,Current Use,Total Assessed Value,ConservativeTax,AllInTax
0,0000000647,Yes,2015,VACANT,550.0,29.00315,59.84165
1,0000000647,Yes,2016,VACANT,550.0,29.00315,59.84165
2,0000000647,Yes,2017,VACANT,550.0,29.00315,59.84165
3,0000000647,Yes,2018,VACANT,550.0,29.00315,59.84165
4,0000000647,Yes,2019,VACANT,550.0,29.00315,59.84165
...,...,...,...,...,...,...,...
24516,0003588939,Yes,2018,VACANT,1000.0,52.73300,108.80300
24517,0003588939,Yes,2019,VACANT,1000.0,52.73300,108.80300
24518,0003588939,Yes,2020,VACANT,1000.0,52.73300,108.80300
24519,0003588939,Yes,2021,VACANT,1000.0,52.73300,108.80300


In [ ]:
df_taxpivotvalues = mergedselect.pivot_table(values=['Total Assessed Value','ConservativeTax','AllInTax'], index='Parcel ID', columns='TAX YEAR', fill_value = 0, aggfunc= ['sum'], margins = True, margins_name='Total')
df_taxpivotvalues

sum                                            \
                AllInTax                                             
TAX YEAR            2015          2016          2017          2018   
Parcel ID                                                            
0000000647      59.84165      59.84165      59.84165      59.84165   
0000003158      54.40150      54.40150      54.40150      54.40150   
0000003247      60.92968      60.92968      30.46484      30.46484   
0000003298      35.90499      35.90499      35.90499      35.90499   
0000003670     108.80300     108.80300     108.80300     108.80300   
...                  ...           ...           ...           ...   
0003261956      23.93666      23.93666      23.93666      23.93666   
0003262073      42.43317      42.43317      42.43317      42.43317   
0003588408      95.74664      95.74664      95.74664      95.74664   
0003588939     108.80300     108.80300     108.80300     108.80300   
Total       161140.50709  160991.44698  160347.33322  159859.89578   

                                                                    \
                                                                     
TAX YEAR            2019          2020          2021          2022   
Parcel ID                                                            
0000000647      59.84165      59.84165      59.84165      59.84165   
0000003158      54.40150      54.40150      54.40150      54.40150   
0000003247      30.46484      30.46484      30.46484      30.46484   
0000003298      35.90499      35.90499      35.90499      35.90499   
0000003670     108.80300     108.80300     108.80300     108.80300   
...                  ...           ...           ...           ...   
0003261956      23.93666      23.93666      23.93666      23.93666   
0003262073      42.43317      42.43317      42.43317      42.43317   
0003588408      95.74664      95.74664      95.74664      95.74664   
0003588939     108.80300     108.80300     108.80300     108.80300   
Total       159607.47282  159009.05632  158702.23186  158555.34781   

                                          ...                \
                         ConservativeTax  ...                 
TAX YEAR           Total            2015  ...         Total   
Parcel ID                                 ...                 
0000000647  4.787332e+02        29.00315  ...     232.02520   
0000003158  4.352120e+02        26.36650  ...     210.93200   
0000003247  3.046484e+02        29.53048  ...     147.65240   
0000003298  2.872399e+02        17.40189  ...     139.21512   
0000003670  8.704240e+02        52.73300  ...     421.86400   
...                  ...             ...  ...           ...   
0003261956  1.914933e+02        11.60126  ...      92.81008   
0003262073  3.394654e+02        20.56587  ...     164.52696   
0003588408  7.659731e+02        46.40504  ...     371.24032   
0003588939  8.704240e+02        52.73300  ...     421.86400   
Total       1.278213e+06     78099.15499  ...  619505.17468   

                                                                              \
           Total Assessed Value                                                
TAX YEAR                   2015     2016     2017     2018     2019     2020   
Parcel ID                                                                      
0000000647                  550      550      550      550      550      550   
0000003158                  500      500      500      500      500      500   
0000003247                  560      560      280      280      280      280   
0000003298                  330      330      330      330      330      330   
0000003670                 1000     1000     1000     1000     1000     1000   
...                         ...      ...      ...      ...      ...      ...   
0003261956                  220      220      220      220      220      220   
0003262073                  390      390      390      390      390      390   
0003588408   

In [ ]:
df_taxpivotvalues.describe()

sum                                               \
               AllInTax                                                
TAX YEAR           2015           2016           2017           2018   
count        604.000000     604.000000     604.000000     604.000000   
mean         533.577838     533.084262     530.951435     529.337403   
std         6573.496901    6567.483485    6541.446068    6521.750213   
min            0.000000       0.000000       0.000000       0.000000   
25%           23.936660      23.936660      23.936660      23.936660   
50%           71.809980      71.809980      71.809980      71.809980   
75%          239.366600     239.366600     239.366600     239.366600   
max       161140.507090  160991.446980  160347.333220  159859.895780   

                                                                      \
                                                                       
TAX YEAR           2019           2020           2021           2022   
count        604.000000     604.000000     604.000000     604.000000   
mean         528.501566     526.520054     525.504079     525.017708   
std         6511.554257    6487.360008    6474.940036    6469.007235   
min            0.000000       0.000000       0.000000       0.000000   
25%           23.936660      23.936660      23.936660      23.936660   
50%           71.809980      71.809980      71.809980      71.809980   
75%          239.366600     239.366600     239.366600     239.366600   
max       159607.472820  159009.056320  158702.231860  158555.347810   

                                        ...                 \
                       ConservativeTax  ...                  
TAX YEAR         Total            2015  ...          Total   
count     6.030000e+02      604.000000  ...     603.000000   
mean      4.239513e+03      258.606473  ...    2054.743531   
std       5.218996e+04     3185.943513  ...   25294.643489   
min       0.000000e+00        0.000000  ...       0.000000   
25%       1.914933e+02       11.601260  ...      92.810080   
50%       5.744798e+02       34.803780  ...     278.430240   
75%       1.914933e+03      116.012600  ...     928.100800   
max       1.278213e+06    78099.154990  ...  619505.174680   

                                                                         \
         Total Assessed Value                                             
TAX YEAR                 2015          2016          2017          2018   
count            6.040000e+02  6.040000e+02  6.040000e+02  6.040000e+02   
mean             4.904073e+03  4.899536e+03  4.879934e+03  4.865099e+03   
std              6.041650e+04  6.036124e+04  6.012193e+04  5.994090e+04   
min              0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%              2.200000e+02  2.200000e+02  2.200000e+02  2.200000e+02   
50%              6.600000e+02  6.600000e+02  6.600000e+02  6.600000e+02   
75%              2.200000e+03  2.200000e+03  2.200000e+03  2.200000e+03   
max              1.481030e+06  1.479660e+06  1.473740e+06  1.469260e+06   

                                                                                
                                                                                
TAX YEAR          2019          2020          2021          2022         Total  
count     6.040000e+02  6.040000e+02  6.040000e+02  6.040000e+02  6.030000e+02  
mean      4.857417e+03  4.839205e+03  4.829868e+03  4.825397e+03  3.896504e+04  
std       5.984719e+04  5.962483e+04  5.951068e+04  5.945615e+04  4.796739e+05  
min       0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  
25%       2.200000e+02  2.200000e+02  2.200000e+02  2.200000e+02  1.760000e+03  
50%       6.600000e+02  6.600000e+02  6.600000e+02  6.600000e+02  5.280000e+03  
75%       2.200000e+03  2.200000e+03  2.200000e+03  2.200000e+03  1.760000e+04  
max       1.466940e+06  1.461440e+06  1.458620e+06  1.457270e+06  1.174796e+07  

[8 rows x 27 columns]